# Real-Time Sign Language Hand Detection

## 1. Business Understanding

### (a) Introduction

In a world where effective communication is the cornerstone of social interaction,there exist significant challenges for individuals with disabilities,particularly those who rely on sign language as their primary means of communication. Communication is not just a basic human right but also a fundamental requirement for participation in society,accessing essential services,and enjoying a high quality of life.

Finger Spelling is a crucial form of communication for individuals with hearing impairments.It involves using handshapes to spell out words and convey messages,making it an essential component of sign language and augmenting other communication methods.For many Deaf and Hard of Hearing individuals,fingerspelling is not just a mode of communication;it is a vital tool for expressing thoughts,emotions and ideas effectively.Despite its significance, there exist a stark communication gap between individuals who rely on fingerspelling and the boarder society.This gap often leads to social isolation,limited access to education and employment opportunities, and barriers to accessing essential services. Bridging this divide is not just a matter of convinience, its a matter of fundamental human rights and social justice.

Recognizing this, the Ministry of Public Services,Gender,Senior Citizens Affairs, and the special Program and Rehabilitative Services Division has embarked on a transformative initiative to break down the barriers faced by individuals with this kind of disability.This division focuses on providing support and services to individuals with special needs. It often encompasses services for people with disabilities, including rehabilitation,vocational training and programs aimed at improving their quality of life.

Our project will harness the power of advanced computer vision techniques to detect and interpret the intricate handshapes and movements that constitute fingerspelling.By doing so in real time, we aim to create a system that can facilitate immediate, meaningful communication between individuals with disabilities and their conterparts.Our vision is to develop an innovative solution that leverages modern technology to empower Deaf and Hard of Hearing individuals by enabling real-time fingerspelling interpretation. This project will improve the ministry's ability to provide services to individuals with disabilities,especially those who rely on fingerspelling.This will enable more effective communication between the government officials and citizens with disabilities,enhancing the quality and accessibility of services.


### (b) The Problem Statement

Effective communication is a fundamental human right and an essential component of social inclusion and participation. However individuals with hearing impairments who rely on fingerspelling as their primary means of communication face often significant communication barriers.The problem at hand is limited accessibility and inclusivity faced by individuals with hearing impairments who rely on fingerspelling.Current communication solutions do not adequately facilitate effective communication between individuals proficient in this visual-gesture language. The Ministry of Public Services,Gender, Senior Citizens Affairs and Special Program and Rehabilitative Services Division have commissioned this project to develop and implement a real-time fingerspelling detection system that can bridge the communication gap between individuals with hearing impairments to communicate effectively with the government officials,access government services, education, and employment opportunities and promote their overall social inclusion and well-being.

### (c) Defining a Metric of Success

- **Recognition Accuracy**

Achieve a recognition accuracy rate of at least 90%, measured by the percentage of correctly identified fingerspelled words out of the total fingerspelled words processed by the system.

- **Response Time**

Ensure a response time of less than 20 seconds for recognizing and translating fingerspelling gestures into text or speech in real-time.

- **Error Rate**

Maintain a low error rate, with recognition errors accounting for less than 5% of total interactions with the system.

- **Intergration success**

Intergrate the system successully into a website,ensuring seamless functionality.

### (d) The Main Objective

To develop and implement a real-time fingerspelling detection system that empowers individuals with hearing impairments to communicate effectively with the broader community,government officials and service providers,thereby promoting inclusivity,accessibility and social intergration.

### (e) Specific Objectives

- **Data Collection**

Collect a diverse and comprehensive dataset of fingerspelling gestures performed by individuals with varying signing styles,hand shapes and speeds

- **Develop a robust hand detection system**

Create an accurate and robust hand detection system using cvzone and MediaPipe to ensure precise tracking of hand movements during fingerspelling.

- **Implement fingerspelling recognition**

Develop a recogntion component that can identify and interpret fingerspelled words and gestures in real-time using machine learning algorithms.

- **Real-time translation**

Intergrate a translation mechanism that converts detected fingerspelled letters into text in real-time,ensuring immediate and accessible communication.

- **User-Friendly Interface**

Design an intuitive user interface that is accessible and user-friendly, allowing individuals with disabilities to interact with the system comfortably

- **Intergration into a website**

Develop a website to intergrate the system to maximize its reach and impact.

### (f) Data Understanding

The data being collected for this project, which comprises a diverse set of fingerspelling gestures performed by two females and two males, holds paramount relevance to the overall success and effectiveness of our fingerspelling recognition system. This dataset encapsulates the rich variability inherent in real-world signing practices, encompassing distinct signing styles, handshapes, and gender-specific variations. By including equal representation from both genders, we acknowledge potential differences in signing articulation, which ensures that our system can proficiently recognize and accommodate diverse signing patterns. Additionally, capturing variations in lighting conditions and backgrounds reflects the environmental diversity that users may encounter, further enhancing the system's robustness and adaptability. In essence, this meticulously collected and annotated dataset serves as the bedrock upon which our project's success is built, facilitating the development of a highly accurate, inclusive, and adaptable real-time fingerspelling recognition system that effectively bridges communication barriers for individuals with hearing impairments.


## 2. Importing Libraries

In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

## 3. Collecting the Data

#### (i) Images

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

offset = 20
imgSize = 300

folder = 'Data/A'
counter = 0

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgwhite = np.ones((imgSize, imgSize, 3), np.uint8)*255

        imgCrop =img[y-offset:y + h+offset, x-offset:x + w+offset]

        imgCropShape = imgCrop.shape

        

        aspectRatio = h/w
        if aspectRatio > 1:
            k = imgSize/h
            wCal = math.ceil(k*w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil(imgSize-wCal)/2
            imgwhite[:, int(wGap) : int(wCal) + int(wGap)] = imgResize

        else:
            k = imgSize/w
            hCal = math.ceil(k*h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))

            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize-hCal)/2)
            imgwhite[int(hGap) : int(hCal) + int(hGap), :] = imgResize


        cv2.imshow('ImageCrop', imgCrop)
        cv2.imshow('ImageWhite', imgwhite)

    cv2.imshow('Image', img)
    key = cv2.waitKey(10)
    if key == ord('s'):
        counter += 1
        cv2.imwrite(f'{folder}/Image_{time.time()}.jpg', imgwhite)
        print(counter)


#### (ii) Videos

In [ ]:
# Creating a folder to save the captured videos if it doesn't exist
output_folder = 'Data/Z/'
os.makedirs(output_folder, exist_ok=True)

# Opening a connection to the webcam (0 for default webcam)
cap = cv2.VideoCapture(0)

# Defining the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Countering to keep track of video file names
video_counter = 10

while True:
    # Generating a new filename for each video
    video_filename = os.path.join(output_folder, f'output_{video_counter}.avi')
    out = cv2.VideoWriter(video_filename, fourcc, 20.0, (frame_width, frame_height))  # Adjust resolution as needed

    # Incrementing the video counter
    video_counter += 1

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Writing the frame to the output video file
        out.write(frame)

        # Displaying the frame
        cv2.imshow('Video', frame)

        key = cv2.waitKey(1)
        if key == ord('q'):  # Press 'q' to stop recording and start a new video
            break

    # Releasing the VideoWriter object for the current video
    out.release()

    key = cv2.waitKey(1)
    if key == ord('q'):  # Press 'q' to stop recording and start a new video
        break

# Releasing the VideoCapture object
cap.release()

# Closing all windows
cv2.destroyAllWindows()
